In [1]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto(
    gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8),
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

In [2]:
import amp.data_utils.data_loader as data_loader
import amp.data_utils.sequence as du_sequence
import numpy as np
import pandas as pd
import sklearn
from amp.config import hydra
from amp.models.decoders import amp_expanded_decoder
from amp.models.encoders import amp_expanded_encoder
from amp.models.master import master
from amp.utils import basic_model_serializer, callback, generator
from keras import backend, layers
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [3]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)

In [4]:
seed = 7
np.random.seed(seed)

In [5]:
from amp.config import MIN_LENGTH, MAX_LENGTH, LATENT_DIM, MIN_KL, RCL_WEIGHT, HIDDEN_DIM, MAX_TEMPERATURE

kl_weight = backend.variable(MIN_KL, name="kl_weight")
tau = backend.variable(MAX_TEMPERATURE, name="temperature")

# Import data

In [6]:
data_manager = data_loader.AMPDataManager(
    '../../data/unlabelled_positive.csv',
    '../../data/unlabelled_negative.csv',
    min_len=MIN_LENGTH,
    max_len=MAX_LENGTH)

amp_x, amp_y = data_manager.get_merged_data()
amp_x_train, amp_x_test, amp_y_train, amp_y_test = train_test_split(amp_x, amp_y, test_size=0.1, random_state=36)
amp_x_train, amp_x_val, amp_y_train, amp_y_val = train_test_split(amp_x_train, amp_y_train, test_size=0.2, random_state=36)

/home/kkrol/hydramp/env/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/kkrol/hydramp/env/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [7]:
# Restrict the length
ecoli_df = pd.read_csv('../../data/mic_data.csv')
mask = (ecoli_df['sequence'].str.len() <= MAX_LENGTH) & (ecoli_df['sequence'].str.len() >= MIN_LENGTH)
ecoli_df = ecoli_df.loc[mask]
mic_x = du_sequence.pad(du_sequence.to_one_hot(ecoli_df['sequence']))
mic_y = ecoli_df.value

In [8]:
mic_x_train, mic_x_test, mic_y_train, mic_y_test = train_test_split(mic_x, mic_y, test_size=0.1, random_state=36)
mic_x_train, mic_x_val, mic_y_train, mic_y_val = train_test_split(mic_x_train, mic_y_train, test_size=0.2, random_state=36)

In [9]:
new_train = pd.DataFrame()
new_train['value'] = list(mic_y_train)
new_train['sequence'] = list(mic_x_train)
active_peptides = new_train[new_train.value < 1.5]
active_peptides  = pd.concat([active_peptides] * 25)
new_train  = pd.concat([
    new_train,
    active_peptides,    
])
mic_x_train = np.array(list(new_train.sequence)).reshape(len(new_train), 25)
mic_y_train = np.array(new_train.value).reshape(len(new_train),)

# Import pretrained classifiers

In [10]:
bms = basic_model_serializer.BasicModelSerializer()
amp_classifier = bms.load_model('../../models/amp_classifier')
amp_classifier_model = amp_classifier()
mic_classifier = bms.load_model('../../models/mic_classifier/')
mic_classifier_model = mic_classifier() 

# Set up cVAE

In [11]:
input_to_encoder = layers.Input(shape=(MAX_LENGTH,))
input_to_decoder = layers.Input(shape=(LATENT_DIM + 2,))

def reset_models():
    global encoder, encoder_model, decoder, decoder_model, master_model, master_keras_model
    
    encoder = amp_expanded_encoder.AMPEncoderFactory.get_default(HIDDEN_DIM, LATENT_DIM, MAX_LENGTH)
    encoder_model = encoder(input_to_encoder)
    decoder = amp_expanded_decoder.AMPDecoderFactory.build_default(LATENT_DIM + 2, tau, MAX_LENGTH)
    decoder_model = decoder(input_to_decoder)
    master_model = master.MasterAMPTrainer(
        amp_classifier=amp_classifier,
        mic_classifier=mic_classifier,
        encoder=encoder,
        decoder=decoder,
        kl_weight=kl_weight,
        rcl_weight=RCL_WEIGHT, 
        master_optimizer=Adam(lr=1e-3),
        loss_weights=hydra,
    )
    master_keras_model = master_model.build(input_shape=(MAX_LENGTH, 21))

In [12]:
reset_models()

tracking <tf.Variable 'temperature:0' shape=() dtype=float32, numpy=2.0> temperature
tracking <tf.Variable 'kl_weight:0' shape=() dtype=float32, numpy=1e-04> kl_weight
Tensor("vae_loss_1/add_1:0", shape=(None,), dtype=float32)


In [13]:
#DATASET_AMP/MIC_TRAIN/VAL

amp_amp_train = amp_classifier_model.predict(amp_x_train, verbose=1, batch_size=10000).reshape(len(amp_x_train))
amp_mic_train = mic_classifier_model.predict(amp_x_train, verbose=1, batch_size=10000).reshape(len(amp_x_train))
amp_amp_val = amp_classifier_model.predict(amp_x_val, verbose=1, batch_size=10000).reshape(len(amp_x_val))
amp_mic_val = mic_classifier_model.predict(amp_x_val, verbose=1, batch_size=10000).reshape(len(amp_x_val))

mic_amp_train = amp_classifier_model.predict(mic_x_train, verbose=1, batch_size=10000).reshape(len(mic_x_train))
mic_mic_train = mic_classifier_model.predict(mic_x_train, verbose=1, batch_size=10000).reshape(len(mic_x_train))
mic_amp_val = amp_classifier_model.predict(mic_x_val, verbose=1, batch_size=10000).reshape(len(mic_x_val))
mic_mic_val = mic_classifier_model.predict(mic_x_val, verbose=1, batch_size=10000).reshape(len(mic_x_val))

uniprot_x_train = np.array(du_sequence.pad(du_sequence.to_one_hot(pd.read_csv('../../data/Uniprot_0_25_train.csv').Sequence)))
uniprot_x_val = np.array(du_sequence.pad(du_sequence.to_one_hot(pd.read_csv('../../data/Uniprot_0_25_val.csv').Sequence)))

uniprot_amp_train = amp_classifier_model.predict(uniprot_x_train, verbose=1, batch_size=10000).reshape(len(uniprot_x_train))
uniprot_mic_train = mic_classifier_model.predict(uniprot_x_train, verbose=1, batch_size=10000).reshape(len(uniprot_x_train))
uniprot_amp_val = amp_classifier_model.predict(uniprot_x_val, verbose=1, batch_size=10000).reshape(len(uniprot_x_val))
uniprot_mic_val = mic_classifier_model.predict(uniprot_x_val, verbose=1, batch_size=10000).reshape(len(uniprot_x_val))


22524/22524 [==============================] - 0s 13us/step


In [14]:
training_generator = generator.concatenated_generator(
    uniprot_x_train,
    uniprot_amp_train,
    uniprot_mic_train,
    amp_x_train,
    amp_amp_train,
    amp_mic_train,
    mic_x_train,
    mic_amp_train,
    mic_mic_train,
    128
)
validation_generator = generator.concatenated_generator(
    uniprot_x_val,
    uniprot_amp_val,
    uniprot_mic_val,
    amp_x_val,
    amp_amp_val,
    amp_mic_val,
    mic_x_val,
    mic_amp_val,
    mic_mic_val,
    128
)

In [15]:
from amp.data_utils.ilso_dataloader import ILSOGenerator

aggregated_amp_train = np.concatenate((amp_amp_train, mic_amp_train, uniprot_amp_train))
aggregated_amp_val = np.concatenate((amp_amp_val, mic_amp_val, uniprot_amp_val))

aggregated_mic_train = np.concatenate((amp_mic_train, mic_mic_train, uniprot_mic_train))
aggregated_mic_val = np.concatenate((amp_mic_val, mic_mic_val, uniprot_mic_val))

training_dataset = ILSOGenerator(
    training_generator, aggregated_amp_train, aggregated_mic_train, 
    [
        "amp_prediction",
        "mic_prediction",
        "vae_loss_1",
        "mic_mean_grad",
        "amp_mean_grad",
        "unconstrained_sleep_mic_output_grad_input",
        "unconstrained_sleep_amp_output_grad_input",
        "correction_sleep_mic_output_grad",
        "correction_sleep_amp_output_grad",
        "correction_sleep_amp_prediction",
        "correction_sleep_mic_prediction",
        "unconstrained_sleep_amp_prediction",
        "unconstrained_sleep_mic_prediction",
        "z_cond_reconstructed_error",
        "correction_sleep_cond_reconstructed_error",
        "unconstrained_sleep_cond_reconstructed_error",
    ], enrichment_size=24)

val_dataset = ILSOGenerator(validation_generator, aggregated_amp_val, aggregated_mic_val) 

In [16]:
from amp.utils.generate_peptides import get_unique, get_z_sigma, single_move_zeros
from amp.data_utils.sequence import pad, to_one_hot

peptides = [
    "ILRWPWWPWRRK", # Omiganan (MBI-226)
    "AKRHHGYKRKFH", # Demegen (P-113)
    "GRRRRSVQWCA", # hLF1-11
    "KNKRKRRRRRRGGRRRR", # Mel4
    "MPKEKVFLKIEKMGRNIRN", # Vishnu3 (HB-107)
]

n = 1000
temp = 2
batch_size=5000


def improve(seq, amp, mic, z_sigma, temp=0.0, batch_size=5000):
    z = encoder_model.predict(seq, batch_size=batch_size)
    noise = np.random.normal(loc=0, scale=temp*z_sigma, size=z.shape)
    encoded = z + noise
    conditioned = np.hstack([
        encoded,
        np.ones((len(seq), 1)),
        np.ones((len(seq), 1)),
    ])
    decoded = decoder_model.predict(conditioned, batch_size=batch_size)
    new_peptides = np.argmax(decoded, axis=2)
    new_peptides = np.array([single_move_zeros(y) for y in new_peptides])
    new_amp = amp_classifier_model.predict(new_peptides, batch_size=batch_size)
    new_mic = mic_classifier_model.predict(new_peptides, batch_size=batch_size)                                                                             
    
    # RELATIVE
    rel_better = new_amp > amp.reshape(-1, 1)
    rel_better = rel_better & (new_mic > mic.reshape(-1, 1))
    rel_better = np.logical_or.reduce(rel_better, axis=1)
    rel_improved = new_peptides[np.where(rel_better), :].reshape(-1, 25)
    before_rel_improve = seq[np.where(rel_better), :].reshape(-1, 25)
    
    # ABSOLUTE
    abs_better = new_amp >= 0.8
    abs_better = abs_better & (new_mic > 0.5)
    abs_better = np.logical_or.reduce(abs_better, axis=1)
    abs_improved = new_peptides[np.where(abs_better), :].reshape(-1, 25)
    before_abs_improve = seq[np.where(abs_better), :].reshape(-1, 25)
    
    return {
        'new_peptides': new_peptides,
        'rel_improved': rel_improved,
        'abs_improved': abs_improved,
        'before_rel_improve': before_rel_improve,
        'before_abs_improve': before_abs_improve, 
        'new_amp': new_amp,
        'new_mic': new_mic,
        }   

def get_new_peptides():
    analogues = []
    new_amps = []
    new_mics = []
    for peptide in peptides:
        pep = pad(to_one_hot([peptide]))
        amp = amp_classifier_model.predict(pep)
        mic = mic_classifier_model.predict(pep)
        z_sigma = get_z_sigma(encoder, pep)

        pep = np.vstack([pep] * n).reshape(-1, 25)
        amp = np.vstack([amp] * n)
        mic = np.vstack([mic] * n)

        analogue_batch = get_unique(improve(pep, amp, mic, z_sigma, temp=temp, batch_size=batch_size)['abs_improved'])
        new_amps.extend(amp_classifier_model.predict(analogue_batch, batch_size=5000))
        new_mics.extend(mic_classifier_model.predict(analogue_batch, batch_size=5000))
        analogues.extend(analogue_batch)
    return analogues, np.array(new_amps).flatten(), np.array(new_mics).flatten()

In [17]:
import keras
class BestGenerationAdder(keras.callbacks.Callback):
    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset

    def on_epoch_end(self, epoch, logs=None):
        seq, amps, mics = get_new_peptides()
        self.dataset.add_examples(seq, amps, mics)

In [18]:
vae_callback = callback.VAECallback(
    encoder=encoder_model,
    decoder=decoder_model,
    tau=tau,
    kl_weight=kl_weight,
    amp_classifier=amp_classifier_model,
    mic_classifier=mic_classifier_model,
)


sm_callback = callback.SaveModelCallback(
    model = master_model,
    model_save_path="../../models/ilso_models/",
    name="HydrAMP_each_epoch")

In [19]:
history = master_keras_model.fit_generator(
    training_dataset,
    steps_per_epoch=1408,
    epochs=11,
    validation_data=validation_generator,
    validation_steps=176,
    callbacks=[vae_callback, sm_callback, BestGenerationAdder(training_dataset)],
)


/home/kkrol/hydramp/env/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


Epoch 1/11
1408/1408 [==============================] - 4439s 3s/step - loss: 3.7285e-04 - amp_prediction_loss: 2.0546e-06 - mic_prediction_loss: 5.4422e-06 - vae_loss_1_loss: 3.7101e-04 - mic_mean_grad_loss: 1.3006e-11 - amp_mean_grad_loss: 1.9442e-09 - unconstrained_sleep_mic_output_grad_input_loss: 9.5623e-10 - unconstrained_sleep_amp_output_grad_input_loss: 5.8347e-08 - correction_sleep_mic_output_grad_loss: 2.8383e-11 - correction_sleep_amp_output_grad_loss: 1.0563e-09 - correction_sleep_amp_prediction_loss: 6.8086e-07 - correction_sleep_mic_prediction_loss: 4.1139e-06 - unconstrained_sleep_amp_prediction_loss: 1.0247e-06 - unconstrained_sleep_mic_prediction_loss: 4.0901e-06 - z_cond_reconstructed_error_loss: 1.3649e-06 - correction_sleep_cond_reconstructed_error_loss: 1.3691e-06 - unconstrained_sleep_cond_reconstructed_error_loss: 7.6075e-06 - amp_prediction_acc: 0.0023 - amp_prediction_binary_crossentropy: 0.8028 - mic_prediction_acc: 0.0000e+00 - mic_prediction_binary_crossentr

Original positive: GGAGHVPEYFVGIGTPISFYG, 
generated: FLLLLLKAKKAKKIKKALK'''''', 
AMP probability: 0.9999936819076538, 
 MIC prediction: 0.9934953451156616. 

Original negative: FPSELANMKNALGFFHIGEIF, 
generated: GAISDLAALLDGLILEIGLIS'''', 
AMP probability: 0.9267773032188416, 
MIC prediction: 1.962632495633443e-06. 

Current KL weight is 0.00010100501
Current temperature is 1.9800997
Epoch 3/11
1408/1408 [==============================] - 3865s 3s/step - loss: 3.0843e-04 - amp_prediction_loss: 1.5279e-06 - mic_prediction_loss: 5.1222e-06 - vae_loss_1_loss: 3.0690e-04 - mic_mean_grad_loss: 1.9794e-10 - amp_mean_grad_loss: 1.3070e-08 - unconstrained_sleep_mic_output_grad_input_loss: 5.1954e-09 - unconstrained_sleep_amp_output_grad_input_loss: 6.7876e-08 - correction_sleep_mic_output_grad_loss: 2.7538e-10 - correction_sleep_amp_output_grad_loss: 1.2563e-08 - correction_sleep_amp_prediction_loss: 1.2512e-06 - correction_sleep_mic_prediction_loss: 3.9261e-06 - unconstrained_sleep_amp_predi

1408/1408 [==============================] - 2454s 2s/step - loss: 2.5923e-04 - amp_prediction_loss: 1.6512e-06 - mic_prediction_loss: 4.1773e-06 - vae_loss_1_loss: 2.5775e-04 - mic_mean_grad_loss: 1.9729e-08 - amp_mean_grad_loss: 5.5971e-08 - unconstrained_sleep_mic_output_grad_input_loss: 2.4078e-08 - unconstrained_sleep_amp_output_grad_input_loss: 5.9034e-08 - correction_sleep_mic_output_grad_loss: 2.2236e-08 - correction_sleep_amp_output_grad_loss: 5.9741e-08 - correction_sleep_amp_prediction_loss: 1.9974e-06 - correction_sleep_mic_prediction_loss: 3.3476e-06 - unconstrained_sleep_amp_prediction_loss: 1.0865e-06 - unconstrained_sleep_mic_prediction_loss: 3.6933e-06 - z_cond_reconstructed_error_loss: 7.8608e-07 - correction_sleep_cond_reconstructed_error_loss: 8.0685e-07 - unconstrained_sleep_cond_reconstructed_error_loss: 5.1026e-06 - amp_prediction_acc: 0.0042 - amp_prediction_binary_crossentropy: 0.6313 - mic_prediction_acc: 0.0000e+00 - mic_prediction_binary_crossentropy: 1.4463

Original positive: GGAGHVPEYFVGIGTPISFYG, 
generated: GFKDLLKGAAKALGKALKF'''''', 
AMP probability: 0.9999983310699463, 
 MIC prediction: 0.9996727705001831. 

Original negative: FPSELANMKNALGFFHIGEIF, 
generated: FPSELSKMKKLLGVVAIIAPF'''', 
AMP probability: 0.001087967655621469, 
MIC prediction: 0.9999715089797974. 

Current KL weight is 0.00011051708
Current temperature is 1.8096749
Epoch 6/11
1408/1408 [==============================] - 2346s 2s/step - loss: 1.1369e-04 - amp_prediction_loss: 1.4188e-06 - mic_prediction_loss: 4.8069e-07 - vae_loss_1_loss: 1.1228e-04 - mic_mean_grad_loss: 2.3019e-07 - amp_mean_grad_loss: 4.3417e-07 - unconstrained_sleep_mic_output_grad_input_loss: 1.8769e-07 - unconstrained_sleep_amp_output_grad_input_loss: 9.3316e-08 - correction_sleep_mic_output_grad_loss: 4.4817e-07 - correction_sleep_amp_output_grad_loss: 5.8827e-07 - correction_sleep_amp_prediction_loss: 4.9050e-06 - correction_sleep_mic_prediction_loss: 6.5744e-06 - unconstrained_sleep_amp_predic

1408/1408 [==============================] - 2347s 2s/step - loss: 8.4189e-05 - amp_prediction_loss: 1.2177e-06 - mic_prediction_loss: 3.6002e-07 - vae_loss_1_loss: 8.2678e-05 - mic_mean_grad_loss: 2.0557e-07 - amp_mean_grad_loss: 5.5054e-07 - unconstrained_sleep_mic_output_grad_input_loss: 2.3894e-07 - unconstrained_sleep_amp_output_grad_input_loss: 1.1044e-07 - correction_sleep_mic_output_grad_loss: 4.5743e-07 - correction_sleep_amp_output_grad_loss: 7.4547e-07 - correction_sleep_amp_prediction_loss: 5.5670e-06 - correction_sleep_mic_prediction_loss: 8.5525e-06 - unconstrained_sleep_amp_prediction_loss: 1.9277e-06 - unconstrained_sleep_mic_prediction_loss: 3.7263e-06 - z_cond_reconstructed_error_loss: 3.1040e-07 - correction_sleep_cond_reconstructed_error_loss: 3.1049e-07 - unconstrained_sleep_cond_reconstructed_error_loss: 3.2773e-06 - amp_prediction_acc: 0.0040 - amp_prediction_binary_crossentropy: 0.4682 - mic_prediction_acc: 0.0000e+00 - mic_prediction_binary_crossentropy: 0.1578

Original positive: GGAGHVPEYFVGIGTPISFYG, 
generated: GFKDLLKGAAKALVKVVLF'''''', 
AMP probability: 0.9999972581863403, 
 MIC prediction: 0.9998102784156799. 

Original negative: FPSELANMKNALGFFHIGEIF, 
generated: FPSELANMKEALGFFHIGEDF'''', 
AMP probability: 0.005511771887540817, 
MIC prediction: 5.271908321446972e-06. 

Current KL weight is 0.00013231297
Current temperature is 1.5115675
Epoch 9/11
1408/1408 [==============================] - 2345s 2s/step - loss: 5.5727e-05 - amp_prediction_loss: 9.7295e-07 - mic_prediction_loss: 2.6109e-07 - vae_loss_1_loss: 5.4149e-05 - mic_mean_grad_loss: 1.9176e-07 - amp_mean_grad_loss: 6.5032e-07 - unconstrained_sleep_mic_output_grad_input_loss: 2.8371e-07 - unconstrained_sleep_amp_output_grad_input_loss: 1.3480e-07 - correction_sleep_mic_output_grad_loss: 4.0799e-07 - correction_sleep_amp_output_grad_loss: 8.4689e-07 - correction_sleep_amp_prediction_loss: 6.2806e-06 - correction_sleep_mic_prediction_loss: 1.0574e-05 - unconstrained_sleep_amp_pre

1408/1408 [==============================] - 2346s 2s/step - loss: 4.9826e-05 - amp_prediction_loss: 9.2391e-07 - mic_prediction_loss: 2.4264e-07 - vae_loss_1_loss: 4.8251e-05 - mic_mean_grad_loss: 1.9306e-07 - amp_mean_grad_loss: 6.7136e-07 - unconstrained_sleep_mic_output_grad_input_loss: 2.9356e-07 - unconstrained_sleep_amp_output_grad_input_loss: 1.4379e-07 - correction_sleep_mic_output_grad_loss: 3.8362e-07 - correction_sleep_amp_output_grad_loss: 8.7050e-07 - correction_sleep_amp_prediction_loss: 6.3599e-06 - correction_sleep_mic_prediction_loss: 1.0965e-05 - unconstrained_sleep_amp_prediction_loss: 1.8419e-06 - unconstrained_sleep_mic_prediction_loss: 2.8024e-06 - z_cond_reconstructed_error_loss: 2.1857e-07 - correction_sleep_cond_reconstructed_error_loss: 2.1824e-07 - unconstrained_sleep_cond_reconstructed_error_loss: 3.1332e-06 - amp_prediction_acc: 0.0039 - amp_prediction_binary_crossentropy: 0.3603 - mic_prediction_acc: 0.0000e+00 - mic_prediction_binary_crossentropy: 0.1068

Original positive: GGAGHVPEYFVGIGTPISFYG, 
generated: GFVILLKGAAKALVKTVLF'''''', 
AMP probability: 0.9999055862426758, 
 MIC prediction: 0.4439588487148285. 

Original negative: FPSELANMKNALGFFHIGEIF, 
generated: FPGDLANMKNALGFFDIGDDF'''', 
AMP probability: 0.0032805451191961765, 
MIC prediction: 2.2464888388640247e-05. 

Current KL weight is 0.00017332529
Current temperature is 1.1538996
